In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env
from pyomo.environ import Constraint
m = wt.watertap_setup(dynamic=False)


wt.case_study_trains.train = {"case_study": "kern_county",
                             "reference": "nawi",
                             "scenario": "wi2"}

wt.case_study_trains.source_water = {"case_study": "kern_county", 
                             "reference": "nawi",
                             "scenario": "baseline",
                             "water_type": ["produced_water", "surface_water"]}

m = wt.case_study_trains.get_case_study(m=m) # flow is set as case study flow unless defined.



kern_county

------- Adding Unit Processes -------
primary_separator
sedimentation
air_flotation
walnut_shell_filter
microfiltration
reverse_osmosis
blending_reservoir
irrigation
landfill
-------------------------------------

adding source
adding source

Connecting unit processes...


In [2]:
wt.display.show_train2(model_name=m)

In [3]:
# RUN MODEL with optimal ro so that the model solves. Then runs again with set pressure. 

# # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
# so that the model solves and gets you results. Then runs again with set pressure.
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("Unfixing feed presure and area for", key, '...\n')
        
        
wt.run_water_tap(m=m, objective=True, skip_small=False)

# prints RO results
print("\nOptimal RO area and pressures:")
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(f'\tFeed pressure for {key}: {round(getattr(m.fs, key).feed.pressure[0](), 2)} bar')
        print(f'\tMembrane area for {key}: {round(getattr(m.fs, key).membrane_area[0](), 2)} m2')

# RESET PRESSURE TO USER INPUT
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        if "feed_pressure" in m.fs.pfd_dict[key]["Parameter"]:
            getattr(m.fs, key).feed.pressure.fix(m.fs.pfd_dict[key]["Parameter"]["feed_pressure"])
            print(f'\nSetting feed presure for {key} to {m.fs.pfd_dict[key]["Parameter"]["feed_pressure"]} bar (user input)...\n')

wt.run_water_tap(m=m, objective=True, print_model_results="summary")

for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(f'\tFeed pressure for {key}: {round(getattr(m.fs, key).feed.pressure[0](), 2)} bar')
        print(f'\tMembrane area: for {key}: {round(getattr(m.fs, key).membrane_area[0](), 2)} m2')


Unfixing feed presure and area for reverse_osmosis ...

Flow is relatively small (< 1 m3/s). Running model with larger dummy flows to initialize...

----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Model finished running to initialize conditions. Now running with actual flow...

----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------

Optimal RO area and pressures:
	Feed pressure for reverse_osmosis: 14.92 bar
	Membrane area for reverse_osmosis: 173586.44 m2

Setting feed presure for reverse_osmosis to 65 bar (user input)...

----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

-----------------------------------------------------------

In [4]:
# set cap utilization factor
m.fs.costing_param.plant_cap_utilization = 1

In [5]:
wt.run_water_tap(m=m, objective=True, print_model_results="summary")
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


PRIMARY SEPARATOR:

	total cap investment: 0.61734
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00819

SEDIMENTATION:

	total cap investment: 0.65594
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00873

AIR FLOTATION:

	total cap investment: 1.42476
	cat and chem cost: 0.0
	electricity cost: 1.22146
	total fixed op cost: 0.01891

WALNUT SHELL FILTER:

	total cap investment: 5.22992
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.06964

MICROFILTRATION:

	total cap investment: 93.10391
	cat and chem cost: 0.0
	electricity cost: 1.23875
	total fixed op cost: 1.23975

REVERSE OSMOSIS:

	total cap investment: 23.9274
	cat and chem cost: 0.22755
	electricity cost: 1.17994
	total fixed op 

In [6]:
# RESET AREA BASED ON RO RECOVERY
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).membrane_area.fix(getattr(m.fs, key).membrane_area[0]())

In [7]:
# Readjust recovery constraint and deactivate objective constraint
m.fs.objective_function.deactivate() # deactivate LCOW objective function

In [8]:
# NOW RUN AS SIMULATION
wt.run_water_tap(m=m, objective=False, print_model_results="summary")

----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


PRIMARY SEPARATOR:

	total cap investment: 0.61734
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00819

SEDIMENTATION:

	total cap investment: 0.65594
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00873

AIR FLOTATION:

	total cap investment: 1.42476
	cat and chem cost: 0.0
	electricity cost: 1.22146
	total fixed op cost: 0.01891

WALNUT SHELL FILTER:

	total cap investment: 5.22992
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.06964

MICROFILTRATION:

	total cap investment: 93.10391
	cat and chem cost: 0.0
	electricity cost: 1.23875
	total fixed op cost: 1.23975

REVERSE OSMOSIS:

	total cap investment: 23.9274
	cat and chem cost: 0.22755
	electricity cost: 1.17994
	total fixed op 

In [9]:
# creates csv in results folder with the name: *case_study*_*scenario*.csv
# In this case, save the final baseline result.

df = wt.get_results_table(m=m, case_study=wt.case_study_trains.source_water["case_study"], 
                                scenario=wt.case_study_trains.source_water["scenario"])